In [1]:
import tensorflow as tf
import os
import gc
import numpy as np 
import pandas as pd
import time
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

SEED = 42

In [2]:
data = np.load('F:/Docs/Repo/Airbus_ship_detection/input/data_for_train.npy')

In [3]:
data_target = np.load('F:/Docs/Repo/Airbus_ship_detection/input/target_for_train.npy')

In [4]:
print("array sizes of data array: ", data.shape)
print("array sizes of target array: ",data_target.shape)
print("example of one image in data array\n", data[0])  
print("example of target for one image in array: ", data_target[0])

array sizes of data array:  (20000, 128, 128, 3)
array sizes of target array:  (20000,)
example of one image in data array
 [[[69 87 61]
  [71 88 61]
  [69 84 59]
  ...
  [69 87 63]
  [64 85 59]
  [66 88 64]]

 [[70 89 62]
  [70 86 60]
  [72 88 63]
  ...
  [70 88 63]
  [66 87 62]
  [65 88 65]]

 [[71 89 62]
  [71 88 61]
  [73 88 63]
  ...
  [70 86 61]
  [69 89 63]
  [67 88 64]]

 ...

 [[79 85 58]
  [84 89 61]
  [82 88 60]
  ...
  [67 85 61]
  [67 85 61]
  [68 86 62]]

 [[81 89 62]
  [79 87 61]
  [76 85 58]
  ...
  [68 86 62]
  [67 85 61]
  [68 86 62]]

 [[79 89 64]
  [76 86 61]
  [77 86 61]
  ...
  [66 84 62]
  [67 85 62]
  [67 85 62]]]
example of target for one image in array:  1


In [5]:
#Set target to one hot target for classification problem
from sklearn.preprocessing import OneHotEncoder
targets = data_target.reshape(len(data_target),-1)
enc = OneHotEncoder()
enc.fit(targets)
targets = enc.transform(targets).toarray()
print(targets.shape)
del data_target
targets

(20000, 2)


array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [6]:
#Split Training data to training data and validate data to detect overfit
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data,targets, test_size = 0.2, random_state = SEED)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((16000, 128, 128, 3), (4000, 128, 128, 3), (16000, 2), (4000, 2))

In [7]:
#Data augumatation
from keras.preprocessing.image import ImageDataGenerator
img_gen = ImageDataGenerator()

In [8]:
#Load ResNet50 model with Keras
#from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
#from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
#from keras.applications.resnet50 import ResNet50 as ResModel
#from keras.applications.vgg16 import VGG16 as VGG16Model
#img_width, img_height = 256, 256
#model = VGG16Model(weights = 'imagenet', include_top=False, input_shape = (img_width, img_height, 3))

In [9]:
gc.collect()

82

In [10]:
#On this case, we only need predict 2 category (1. have ship, 2. no ship)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.callbacks import ModelCheckpoint


# creating the final model 
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (128, 128, 3))) #1
model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (128, 128, 64))) #2
model.add(MaxPool2D(pool_size=(2, 2)))                                                                                  #3
model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (128/2, 128/2, 64))) #4
model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (128/2, 128/2, 128))) #5
model.add(MaxPool2D(pool_size=(2, 2)))                                                                                       #6
model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (64/2, 64/2, 128))) #7
model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (64/2, 64/2, 256))) #8
model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (64/2, 64/2, 256))) #9
model.add(MaxPool2D(pool_size=(2, 2)))                                                                                   #10
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (32/2, 32/2, 256))) #11
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (32/2, 32/2, 512))) #12
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (32/2, 32/2, 512))) #13
model.add(MaxPool2D(pool_size=(2, 2)))                                                                                   #14
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (16/2, 16/2, 512))) #15
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (16/2, 16/2, 512))) #16
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'Same', activation ='relu', input_shape = (16/2, 16/2, 512))) #17
model.add(MaxPool2D(pool_size=(2, 2)))                                                                                    #18

model.add(Flatten())
model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1024, activation = "relu"))
model.add(Dense(2, activation = "softmax"))

filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

image_file = 'model_1.png'
tf.keras.utils.plot_model(model, to_file = image_file, show_shapes = True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [11]:
#Set Hyperparameter and Start training
from keras import optimizers
from keras.optimizers import RMSprop
epochs = 50
lrate = 0.001 #learning rate
batch_size = int(len(x_train) / 9)
decay = lrate/epochs # Learning rate decay over each update
optimizer = RMSprop(lr = 0.001, rho = 0.9, epsilon = 1e-08, decay = decay)
#model.load_weights("transfer_ship_v1_5.h5")
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy', tf.keras.metrics.AUC(curve = 'ROC'), tf.keras.metrics.AUC(curve = 'PR')])
#model_final.summary()

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 256)       2

In [13]:
history = model.fit_generator(img_gen.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, validation_data = (x_val,y_val),
                          steps_per_epoch = int(len(x_train)/batch_size), callbacks=callbacks_list)
model.save('transfer_ship_exp.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50


ResourceExhaustedError:  OOM when allocating tensor with shape[1777,128,128,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node sequential/conv2d/Relu (defined at <ipython-input-13-15c049b5a07a>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_3198]

Function call stack:
train_function


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], color='b', label="Training loss")
plt.plot(history.history['val_loss'], color='r', label="validation loss")
plt.legend(loc='best', shadow=True)
plt.show()

plt.plot(history.history['loss'], color='b', label="Training loss")
plt.plot(history.history['val_loss'], color='r', label="validation loss")
plt.legend(loc='best', shadow=True)
plt.ylim(0, 50)
plt.show()

plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = plt.legend(loc='best', shadow=True)
plt.show()

plt.plot(history.history['auc'], color='b', label="Training ROC")
plt.plot(history.history['val_auc'], color='r',label="Validation ROC")
plt.legend(loc='best', shadow=True)
plt.show()

plt.plot(history.history['auc_1'], color='b', label="Training PR")
plt.plot(history.history['val_auc_1'], color='r',label="Validation PR")
plt.legend(loc='best', shadow=True)
plt.show()

In [ ]:
gc.collect()